# Google Eart Engine for Advanced Geospatial Analisys
### Hydrology - project assignment by L. Carlassara, A. Iseni, E. Poli @ PoliMI
1. Import a watershed of interest as an asset.
2. Export (i.e. download) a DEM, a mean NDVI, a mean precipitation from CHIRPS
and a mean temperature from ERA for the watershed.
3. Compute the forest area in the watershed with GFC hansen, compute the population
inside with JRC settlements, compute the agricultural area using maryland product
for agriculture.

Import libraries

In [1]:
import geemap
import ee

Autenthication with a Google account:

In [2]:
ee.Authenticate()


Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
Map = geemap.Map(center=[46.07,11.12], zoom=8)

Upload the watershed of Adige river 

In [5]:
wrs_shp = ee.FeatureCollection("projects/ee-lorenzocarlassara/assets/bacini")

wrs_adige = wrs_shp.filter(ee.Filter.eq('CODICE', 'N001'))
aoi = wrs_adige.geometry()
Map.addLayer(aoi)
# period of interest
start = '2003-03-01'
end =   '2023-03-31'

### DEM
Compute the Digital Surface Model from Copernicus DEM 

In [6]:
DEM30m = ee.ImageCollection("COPERNICUS/DEM/GLO30")

DEM30m = DEM30m.select('DEM')
terrain = ee.Algorithms.Terrain(DEM30m)
slope = terrain.select('slope')
DEM_wrs_density = {'min': 0, 'max': 4000}
Map.addLayer(DEM30m,DEM_wrs_density,'DEM')           

### NDVI
Compute the NDVI using bands Red (B4) and Near infrared (B5) from Landsdat 8 Collection 2 Tier 1

In [7]:
def addNDVI(input):
    ndvi = input.normalizedDifference(['B5', 'B4']).rename("ndvi")
    return input.addBands(ndvi)

landsat_img_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_RT_TOA")
landsat_img_collection = landsat_img_collection.filterBounds(aoi).filterDate(start,end).filter(ee.Filter.lt('CLOUD_COVER_LAND',50))

ndviCollection = landsat_img_collection.map(addNDVI).mean().clip(aoi)   
ndviCollection_density = {'bands':['ndvi'], min:-1, max:1, 'palette': ['blue', 'white', 'green']}
Map.addLayer(ndviCollection, ndviCollection_density, 'NDVI')

### CHIRPS
Compute the mean of the last 20 years **precitation** from *Climate Hazards Group InfraRed Precipitation with Station* data (CHIRPS)

In [8]:
Rain = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")

Rain20_wrs = Rain.filterDate(start,end).filterBounds(aoi)
Rain20_wrs_mean = Rain20_wrs.mean().clip(aoi)             
Rain20_wrs_density = {'min':1, 'max':5, 'palette': ['red','green','blue']}
Map.addLayer(Rain20_wrs_mean,Rain20_wrs_density,'CHIRPS Precipitation')

### ERA5
Compute the mean of the last 20 years **temperature** from *ECMWF atmospheric reanalysis of the global climate* (ERA5)

In [1]:
Era5Month = ee.ImageCollection("ECMWF/ERA5/MONTHLY")

Era5Temp = Era5Month.select('mean_2m_air_temperature')
Era5Temp = Era5Temp.filterDate(start,end)
Era5Temp_20y_wrs = Era5Temp.filterBounds(aoi)
Era5Temp_20y_wrs_mean = Era5Temp_20y_wrs.mean().clip(aoi)           
Era5Temp_density = {'min': 267, 'max': 287, 'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']}
Map.addLayer(Era5Temp_20y_wrs_mean,Era5Temp_density, 'Era5 Temperature')
Map

NameError: name 'ee' is not defined

### Forest
Compute the **forest area** in the watershed with the *Hansen Global Forest Change* analysis of Landsat images (GFC)

In [ ]:
gfc = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")

treecover = gfc.select(['treecover2000'])              

treecover21 = treecover.updateMask(gfc.select(['loss']).eq(0))

#Map.addLayer(treecover21,{min:0, max:100},'tc21')

#from tree cover (0-100) to forest cover (0-1)
FC = treecover21.gte(10)
#Map.addLayer(FC,{min:0, max:1},'FC')

# compute the area per pixel
FC_area = FC.multiply(ee.Image.pixelArea())
# Sum the values of loss pixels in the watershed area.
treecover_area = FC_area.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': aoi,
  'scale': 30,
  'maxPixels': 1e13
})
# convert area to hectares
treecover_area_ha = treecover_area.getNumber('treecover2000').divide(10000).round()

### Population
Compute the **population** inside the watershed with *Global Human Settlement Layers* (GHSL)

In [ ]:
population_count = ee.Image('JRC/GHSL/P2016/POP_GPW_GLOBE_V1/2015').clip(aoi)

# Calculate the amount of exposed population
# get GHSL projection
GHSLprojection = population_count.projection()

# Create a raster showing exposed population only using the  flood layer
population = population_count

#Sum pixel values of exposed population raster 
stats = population.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': aoi,
  'scale': 250,
  'maxPixels':1e9 
})

# get number of people as integer
number_pp = stats.getNumber('population_count').round()


### Agricolture
Compute the **agricultural area** using maryland product for agriculture by *P. Potapov - Global cropland*

In [ ]:
LC = ee.ImageCollection('users/potapovpeter/Global_cropland_2019').first().clip(aoi)
  
# Extract only cropland pixels using the classes cropland
cropmask = LC.eq(1)
 
cropland = LC.updateMask(cropmask)
  

# get pixel area of affected cropland layer
crop_pixelarea = cropland.multiply(ee.Image.pixelArea()) #calcuate the area of each pixel 

# sum pixels of affected cropland layer
crop_stats = crop_pixelarea.reduceRegion(**{
  'reducer': ee.Reducer.sum(), #sum all pixels with area information                
  'geometry': aoi,
  'scale': 30,
  'maxPixels': 1e9
  })
  
crop_area_ha = crop_stats.getNumber('b1').divide(10000).round()